In [2]:
from nltk.tokenize import RegexpTokenizer
from nltk import pos_tag, word_tokenize
from nltk.corpus import wordnet
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC, LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn import metrics

import numpy as np
import pandas as pd
import time

import progressbar

In [3]:
train = pd.read_csv('./preprocessed/train.csv')
test = pd.read_csv('./preprocessed/test.csv')
test_labels = pd.read_csv('./dataset/test_labels.csv')

print(train.shape)
print(test.shape)
print(test_labels.shape)

(159571, 10)
(153164, 4)
(153164, 7)


In [4]:
train.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate,comment_lemma,comment_stem
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0,Explanation Why the edits make under my userna...,explan whi the edit made under my usernam hard...
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0,D'aww ! He match this background colour I 'm s...,d'aww ! He match thi background colour I 'm se...
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0,"Hey man , I 'm really not try to edit war . It...","hey man , I 'm realli not tri to edit war . It..."
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0,`` More I ca n't make any real suggestion on i...,`` more I ca n't make ani real suggest on impr...
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0,"You , sir , be my hero . Any chance you rememb...","you , sir , are my hero . ani chanc you rememb..."


In [5]:
test.head()

,id,comment_text,comment_lemma,comment_stem
0,00001cee341fdb12,Yo bitch Ja Rule is more succesful then you'll...,Yo bitch Ja Rule be more succesful then you 'l...,Yo bitch Ja rule is more succes then you 'll e...
1,0000247867823ef7,== From RfC == \n\n The title is fine as it is...,"== From RfC == The title be fine as it be , IMO .","== from rfc == the titl is fine as it is , imo ."
2,00013b17ad220c46,""" \n\n == Sources == \n\n * Zawe Ashton on Lap...",`` == Sources == * Zawe Ashton on Lapland — / ``,`` == sourc == * zaw ashton on lapland — / ``
3,00017563c3f7919a,":If you have a look back at the source, the in...",": If you have a look back at the source , the ...",": If you have a look back at the sourc , the i..."
4,00017695ad8997eb,I don't anonymously edit articles at all.,I do n't anonymously edit article at all .,I do n't anonym edit articl at all .


In [6]:
token_regex = r'[a-zA-Z0-9]+'
tokenizer = RegexpTokenizer(token_regex).tokenize

In [7]:
def training(X, classifier, **kwargs):
    test_cols = [ 'toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']
    roc_aoc_scores = []
    clfs = []
    
    for eval_col in test_cols:
#         print("FIT ", eval_col)
        y = train[eval_col]
        X_train, X_test, y_train, y_test = train_test_split(X, y,  test_size=0.3, random_state=101)

        clf = classifier(**kwargs).fit(X_train, y_train)
        predicted= clf.predict_proba(X_test)[:,1]
            
        roc_aoc_scores.append(metrics.roc_auc_score(y_test, predicted))
        clfs.append(clf)
    
    avg_score = np.mean(np.array(roc_aoc_scores))
    print("Score :" , avg_score)
    return clfs, avg_score

In [68]:
cv_default_params = {
    "lowercase" : True,
    "stop_words" : 'english',
    "ngram_range" : (1,1) ,
    "tokenizer" : tokenizer
}

cv_strip_params = cv_default_params.copy()
cv_strip_params["ngram_range"] = (1,2)
cv_strip_params["max_features"] = 80000


params_options = [cv_default_params, cv_strip_params]
params_text = ['DEFUALT', 'STRIP']

text_options = ['comment_text']

classifiers = [clf]
classifiers_text = ['LR']

params_for_LR = {"max_iter" : 1600}
classifiers_params = [params_for_LR]

In [70]:
for i, params in enumerate(params_options):
    for j, texts in enumerate(text_options):
        print("Result for {:>8} params with {:>15} as input".format(
            params_text[i], text_options[j]
        ))
        cv = CountVectorizer(**params)
        text_cv = cv.fit_transform(train[texts])

        for k, classifier in enumerate(classifiers):
            print("Classfier {:>10}".format(classifiers_text[j]))
            trained_clfs, score = training(text_cv, classifier, **classifiers_params[k])
            
            
            
        
        

Result for  DEFUALT params with    comment_text as input
Classfier         LR
Score : 0.9355201972913837
Result for    STRIP params with    comment_text as input
Classfier         LR


KeyboardInterrupt: 

In [9]:
vectorizer = {
    'cv' : CountVectorizer(lowercase=False, stop_words='english', ngram_range = (1,1), tokenizer = tokenizer),
    'cv_max' : CountVectorizer(lowercase=False, stop_words='english', ngram_range = (1,2), tokenizer = tokenizer, max_features=80000),
    
    'tf' : TfidfVectorizer(lowercase=True, strip_accents='unicode', ngram_range = (1,1), tokenizer = tokenizer),
    'tf_max' : TfidfVectorizer(lowercase=True, strip_accents='unicode', ngram_range = (1,2), tokenizer = tokenizer, max_features=100000)
}

In [ ]:
text_vec = {}

for v in vectorizer:
    

In [42]:
cv = CountVectorizer(lowercase=False, stop_words='english', ngram_range = (1,1), tokenizer = tokenizer)
text_cv = cv.fit_transform(train['comment_text'])

In [71]:
num_features = len(cv.get_feature_names())
print("Input Name   : ", "Simple CV")
print("Num Features : ", num_features)

Input Name   :  Simple CV
Num Features :  227490


In [72]:
cv_lemma = CountVectorizer(lowercase=False, stop_words='english', ngram_range = (1,1), tokenizer = tokenizer)
text_cv_lemma = cv_lemma.fit_transform(train['comment_lemma'])

In [73]:
num_features = len(cv_lemma.get_feature_names())
print("Input Name   : ", "Lemma CV")
print("Num Features : ", num_features)

Input Name   :  Lemma CV
Num Features :  218494


In [78]:
cv_stem = CountVectorizer(lowercase=False, stop_words='english', ngram_range = (1,1), tokenizer = tokenizer)
text_cv_stem = cv_stem.fit_transform(train['comment_stem'])

In [79]:
num_features = len(cv_stem.get_feature_names())
print("Input Name   : ", "Stem CV")
print("Num Features : ", num_features)

Input Name   :  Stem CV
Num Features :  154983


In [91]:
cv_strip = CountVectorizer(lowercase=False, stop_words='english', ngram_range = (1,2), tokenizer = tokenizer, max_features=80000)
text_cv_strip = cv_strip.fit_transform(train['comment_text'])

In [92]:
num_features = len(cv_strip.get_feature_names())
print("Input Name   : ", "Strip CV")
print("Num Features : ", num_features)

Input Name   :  Strip CV
Num Features :  80000


In [12]:
cv_strip_1 = CountVectorizer(lowercase=False, stop_words='english', ngram_range = (1,3), tokenizer = tokenizer, max_features=100000)
text_cv_strip_1 = cv_strip_1.fit_transform(train['comment_text'])

In [13]:
num_features = len(cv_strip_1.get_feature_names())
print("Input Name   : ", "Strip 1 CV")
print("Num Features : ", num_features)

Input Name   :  Strip 1 CV
Num Features :  100000


In [14]:
cv_strip_lemma = CountVectorizer(lowercase=False, stop_words='english', ngram_range = (1,2), tokenizer = tokenizer, max_features=80000)
text_cv_strip_lemma = cv_strip_lemma.fit_transform(train['comment_lemma'])

In [15]:
num_features = len(cv_strip_lemma.get_feature_names())
print("Input Name   : ", "Strip Lemma CV")
print("Num Features : ", num_features)

Input Name   :  Strip Lemma CV
Num Features :  80000


In [9]:
tf_simple = TfidfVectorizer(lowercase=True, strip_accents='unicode', ngram_range = (1,2), tokenizer = tokenizer, max_features=100000)
text_tf_simple = tf_simple.fit_transform(train['comment_lemma'][:100000])

In [10]:
num_features = len(tf_simple.get_feature_names())
print("Input Name   : ", "Simple TFIDF")
print("Num Features : ", num_features)

Input Name   :  Simple TFIDF
Num Features :  100000


### Classifiers

In [10]:
def clf(**kwargs):
    return LogisticRegression(C=2, solver='lbfgs', **kwargs)

In [12]:
from sklearn.svm import SVC

def svm(**kwargs):
    return SVC(**kwargs)

#### Results

In [70]:
trained_clfs, score = training(text_cv, clf, max_iter=1600)

Score : 0.9417094887931721


In [75]:
trained_clfs, score = training(text_cv_lemma, clf, max_iter=1600)

Score : 0.9425879191164225


In [80]:
trained_clfs, score = training(text_cv_stem, clf, max_iter=1600)

Score : 0.9364430774362767


In [93]:
trained_clfs, score = training(text_cv_strip, clf, max_iter=1600)

Score : 0.9471315301415034


In [103]:
trained_clfs, score = training(text_cv_strip_1, clf, max_iter=1600)
# For NGRAM 1,3 , Max Features 1000

Score : 0.9470295766696221


In [11]:
trained_clfs, score = training(text_cv_strip_lemma, clf, max_iter=1600)

Score : 0.947878680776361


In [ ]:
trained_clfs, score = training(text_tf_simple, svm, kernel='linear')

In [18]:
len(train['comment_text'])

159571

In [ ]:
tf = TfidfVectorizer(ngram_range = (1,1),  strip_accents='unicode', tokenizer = tokenizer, max_features=80000)